<a id='pred_top'>

# Predict auction price

Try several models and improve predicition accuracy

## Model fitting

- Linear fits  
  1. [Simple linear fit](#pred_model_1)  
     No cross validation. Observations with missing values are dropped.
  2. [Dependent values scaled](#pred_model_2)  
     Dependent value here is _prices_.
  3. [Partial data](#pred_model_3)  
     Only young cars
- Multiple linear regression models  
  1. [MLR fit without imputation](#pred_model_4)  
  2. [With imputation](#pred_model_5)  
  3. [Include categorical features](#pred_model_6)  
  4. [Lasso regularization](#pred_model_7)  
  5. [include engineered features](#pred_model_8)

## Results

- [Model performance](#pred_accuracies)
- [Save best model](#pred_save_model) **TODO**  
  This is not implemented yet. Some preprocessing functions are not handled well with `pickle`.
- [Predictions](#pred_predict)
     
  

In [1]:
import sys
import os
import re
import json

In [2]:
with open('../assets/drz-settings-current.json', 'r') as fid:
    cfg = json.load(fid)
print(cfg['AUCTION'])

if cfg['AUCTION']['kind'] == 'opbod':
    raise NotImplementedError
    
OPBOD = cfg['AUCTION']['kind'] == 'opbod'
AUCTION_ID = cfg['AUCTION']['id']
DATA_DIR = cfg['FILE_LOCATION']['data_dir']
RESULTS_DIR = cfg['FILE_LOCATION']['report_dir']
VERBOSE = int(cfg['GENERAL']['verbose'])
SAVE_METHOD = cfg['GENERAL']['save_method']


{'kind': 'inschrijving', 'id': '2024-0006', 'date': '20240323'}


In [3]:
if SAVE_METHOD == 'skip_when_exist':
    do_save = lambda fn: not(os.path.isfile(fn))
elif SAVE_METHOD == 'always_overwrite':
    do_save = lambda _: True
elif SAVE_METHOD == 'skip_save':
    do_save = lambda _: False
else:
    raise NotImplementedError(f'SAVE_METHOD: {SAVE_METHOD} not implemented')

In [4]:
SAVE_METHOD

'skip_when_exist'

In [5]:
TAG_SINGLE = "nbconvert_instruction:remove_single_output"


In [6]:
# import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

import seaborn as sns

In [7]:
# set figure defaults (needs to be in cell seperate from import sns)
plt.style.use([
    'default',
    f"{cfg['FILE_LOCATION']['app_dir']}/assets/movshon.mplstyle",
    f"{cfg['FILE_LOCATION']['app_dir']}/assets/context-notebook.mplstyle"
])

# Load data

In [8]:
fn = f'{DATA_DIR}/cars-for-ml.pkl'
print(fn)
df = pd.read_pickle(fn)
print(df.shape)

# time deltas
sel = df.dtypes == 'timedelta64[ns]'
df.loc[:, sel] = df.loc[:, sel].applymap(lambda x: x.days)
# nullable boolean
sel = df.dtypes == 'boolean'
df.loc[:,sel] = df.loc[:,sel].astype('O').fillna(np.NaN)
# int to float
df.price = df.price.astype(float)
# categories
cat_columns = ['brand', 'model', 'fuel', 'body_type','color', 'energy_label', 'fourwd', 'automatic_gearbox', 'under_survey']
# numerical
num_columns = list(np.setdiff1d(df.columns, cat_columns + ['price']))
df.loc[:, num_columns] = df.loc[:, num_columns].astype(float)

# Factorized categorical values
fld = 'energy_label'
# replace empty with NaN creates factor '-1'
v, idx = pd.factorize(df[fld].replace({'': np.NaN}), sort=True)
# convert '-1' back to NaN
v = v.astype(float)
v[v==-1] = np.NaN
# Store in dataframe
new_col = 'converted_' + fld
df[new_col] = v
# update list
num_columns += [new_col]
cat_columns.remove(fld)
print('\nCategorical field [{}] is converted to sequential numbers with: '.format(fld), end='\n\t')
print(*['{} <'.format(c) for c in idx], end='\n\n')

# convert boolean to string
for fld in ['fourwd', 'automatic_gearbox', 'under_survey']:
    if fld not in df.columns:
        print(f'!{fld} not in data!. Skip for now')
        continue
    new_col = fld
    # # update list
    # cat_columns += [new_col]
    # cat_columns.remove(fld)
    replace_dict = {
        '': '', 
        True: 'y', 
        False: 'n'
    }
    df[new_col] = df[fld].replace(replace_dict)
    print('\nBoolean field [{}] is converted to numbers according to: '.format(fld), end='\n')
    print(*['\t"{}" -> {} ({})\n'.format(k,v, type(v)) for k,v in replace_dict.items()], end='\n\n')

# convert integer to float and replace -1
for fld in ['number_of_cylinders', 'number_of_doors', 'number_of_gears', 'number_of_seats']:
    if fld not in df.columns:
        print(f'!{fld} not in data!. Skip for now')
        continue
    new_col = fld
    replace_dict = {
        -1: np.NaN, 
    }
    df[new_col] = df[fld].replace(replace_dict).astype(float)

# convert empty string to NaN
for fld in ['brand', 'model', 'fuel', 'body_type', 'color', 'fourwd']:
    if fld not in df.columns:
        print(f'!{fld} not in data!. Skip for now')
        continue
    new_col = fld
    replace_dict = {
        '': np.NaN, 
    }
    df[new_col] = df[fld].replace(replace_dict)

# translate Dutch to English
fld = 'color'
new_col = fld
# # update list
# cat_columns += [new_col]
# cat_columns.remove(fld)
replace_dict = {
    '': 'missing', 
    'BLAUW': 'Blue',
    'ROOD': 'Red',
    'GROEN': 'Green',
    'GRIJS': 'Gray',
    'WIT': 'White',
    'ZWART': 'Black',
    'BEIGE': 'Beige',
    'BRUIN': 'Brown',
    'ROSE': 'Pink',
    'GEEL': 'Yellow',
    'CREME': 'Creme',
    'ORANJE': 'Orange',
    'PAARS': 'Purple,'
}
df[new_col] = df[fld].replace(replace_dict)
print('\nField [{}] is converted according to: '.format(fld), end='\n')
print(*['\t"{}" -> {} ({})\n'.format(k,v, type(v)) for k,v in replace_dict.items()], end='\n\n')

# reporting
try:
    print('Categorical:', len(cat_columns))
    [print('\t[{:2.0f}] {:s}'.format(i+1, c)) for i,c in enumerate(df[cat_columns].columns)]
    print('Numercial:', len(num_columns))
    [print('\t[{:2.0f}] {:s}'.format(i+1, c)) for i,c in enumerate(df[num_columns].columns)]
    print('Last lot in data set:\n\t{}'.format(df.index[-1]))
except:
    cat_columns = [c for c in cat_columns if c in df.columns]
    num_columns = [c for c in num_columns if c in df.columns]    
    print('! not all fields are in data !. Skip for now')

/home/tom/bin/satdatsci/Saturday-Datascience/data/cars-for-ml.pkl
(11574, 29)

Categorical field [energy_label] is converted to sequential numbers with: 
	A < B < C < D < E < F < G < nan <


Boolean field [fourwd] is converted to numbers according to: 
	"" ->  (<class 'str'>)
 	"True" -> y (<class 'str'>)
 	"False" -> n (<class 'str'>)



Boolean field [automatic_gearbox] is converted to numbers according to: 
	"" ->  (<class 'str'>)
 	"True" -> y (<class 'str'>)
 	"False" -> n (<class 'str'>)



Boolean field [under_survey] is converted to numbers according to: 
	"" ->  (<class 'str'>)
 	"True" -> y (<class 'str'>)
 	"False" -> n (<class 'str'>)



Field [color] is converted according to: 
	"" -> missing (<class 'str'>)
 	"BLAUW" -> Blue (<class 'str'>)
 	"ROOD" -> Red (<class 'str'>)
 	"GROEN" -> Green (<class 'str'>)
 	"GRIJS" -> Gray (<class 'str'>)
 	"WIT" -> White (<class 'str'>)
 	"ZWART" -> Black (<class 'str'>)
 	"BEIGE" -> Beige (<class 'str'>)
 	"BRUIN" -> Brown (<class 'str

In [9]:
# Store model results in dictonary: Instantiate empty dict
models = dict()

In [10]:
def split_shelve_vars():
    import types
    to_shelve = {}
    not_to_shelve = {}

    # loop over global variables (within this function is ignored)
    for var,val in globals().items():
        
        # skip variables based on names
        if re.match('^_(\d+|(i+\d*))$', var) is not None:
            not_to_shelve[var] = '-n'
            continue
        if re.match('^_+$', var) is not None:
            not_to_shelve[var] = '---'
            continue
        if var in ('_dh', '_ih', '_oh'):
            not_to_shelve[var] = '-dio'
            continue
        if var in ('In', 'Out'):
            not_to_shelve[var] = '-io'
            continue
        if var in ('__doc__', '__loader__', '__name__', '__package__', '__session__', '__spec__'):
            not_to_shelve[var] = val
            continue
        
        # skip built-ins and modules
        if isinstance(globals()[var], (types.ModuleType, types.BuiltinFunctionType, types.FunctionType)):
            not_to_shelve[var] = type(globals()[var])
            continue
        else:
            pass
            #print(globals()[var].__class__, var)

        # store
        to_shelve[var] = val
        
    return not_to_shelve, to_shelve
    
drop,keep = split_shelve_vars()
list(keep.keys())

['get_ipython',
 'exit',
 'quit',
 'fid',
 'cfg',
 'OPBOD',
 'AUCTION_ID',
 'DATA_DIR',
 'RESULTS_DIR',
 'VERBOSE',
 'SAVE_METHOD',
 'TAG_SINGLE',
 'fn',
 'df',
 'sel',
 'cat_columns',
 'num_columns',
 'fld',
 'v',
 'idx',
 'new_col',
 'replace_dict',
 'models']

In [11]:
keep.pop('get_ipython')
keep.pop('exit')
keep.pop('quit')
keep.pop('fid')

<_io.TextIOWrapper name='../assets/drz-settings-current.json' mode='r' encoding='UTF-8'>

In [12]:
import shelve
from inspect import getsource
import types


In [13]:
with shelve.open('./predict-price.shelve', flag='n') as slf:
    for k in [
        'cfg',
        #'OPBOD',
        #'AUCTION_ID',
        #'DATA_DIR',
        'RESULTS_DIR',
        #'VERBOSE',
        #'SAVE_METHOD',
        #'TAG_SINGLE',
        #'fn',
        'df',
        'cat_columns',
        'num_columns',
        'models',
        'do_save']:
        print(k)
        if k in keep:
            v = keep[k]
        else:
            v = globals()[k]
            if isinstance(v, types.FunctionType):
                src = getsource(v).strip()
                cnt = len([k for k in slf.keys() if k.startswith('def')])
                k = f'def{cnt}:'
                v = src
            
        slf[k] = v


cfg
RESULTS_DIR
df
cat_columns
num_columns
models
do_save


In [14]:
for m in range(1,9):
    %run ./predict-price-model{m}.ipynb

RESULTS_DIR
def0:
do_save = lambda fn: not(os.path.isfile(fn))
cfg
num_columns
cat_columns
models
df
(8491, 1)
(8491, 1)
/home/tom/bin/satdatsci/Saturday-Datascience/results/linear_regression_no_cv.png
Shelve file [./predict-price.shelve] contains models:
	linear regression no cv
RESULTS_DIR
def0:
do_save = lambda fn: not(os.path.isfile(fn))
cfg
num_columns
cat_columns
models
df
(8308, 1)
(8308, 1)
(5815, 1)
(2493, 1)
According to "linear regression log price young"-model
Car depreciates to half its value every
	1332 days (3.6 years).
	y(t=   +0) = 26833 euro
	y(t=   +2) = 18346 euro
	y(t=   +4) = 12543 euro
	y(t=   +6) = 8576 euro
	y(t=   +8) = 5863 euro

	y(t= +3.6) = 13417 euro
	y(t=0) / 2 = 13417 euro
/home/tom/bin/satdatsci/Saturday-Datascience/results/linear_regression_log_price_young.png
Shelve file [./predict-price.shelve] contains models:
	linear regression no cv
	linear regression log price young
RESULTS_DIR
def0:
do_save = lambda fn: not(os.path.isfile(fn))
cfg
num_columns
c

AssertionError: stop running, below is sandboxing and testing

AssertionError: stop running, below is sandboxing and testing

In [15]:
with shelve.open('./predict-price.shelve', flag='r') as slf:
    for k,v in slf.items():
        print(k)
        globals()[k] = v
        if re.match('def\d+:', k) is not None:
            print(v)
            exec(v) 

RESULTS_DIR
def0:
do_save = lambda fn: not(os.path.isfile(fn))
cfg
num_columns
cat_columns
models
df


In [16]:
# Best model
model_name = 'MLR added features'

# Display prediction errors

x_sample = df.dropna(subset=['price']).iloc[:,1:]
y_sample = df.dropna(subset=['price']).iloc[:,0]
# Add features
x_sample.loc[:,'usage_intensity'] = x_sample.odometer / x_sample.age
x_sample.loc[:,'classic'] = x_sample.age > 25*365
x_sample.loc[:,'classic'].replace({True:'y', False:'n'}, inplace=True)
#x_sample.loc[:,'youngtimer'] = (x_sample.age > 15*365) & (x_sample.age <= 25*365)
#x_sample.loc[:,'youngtimer'].replace({True:'y', False:'n'}, inplace=True)
# predict again
y_sample_pred = models[model_name]['model'].predict(x_sample) 

x_sample['price'] = y_sample
x_sample['prediction_error'] = y_sample_pred - y_sample
x_sample['prediction_error_fraction'] = y_sample_pred/y_sample
x_sample['prediction_error_log'] = np.log10(x_sample.prediction_error_fraction)
x_sample['prediction_error_abslog'] = np.abs(np.log10(x_sample.prediction_error_fraction))
x_sample['prediction'] = y_sample_pred
x_sample['age_y'] = x_sample.age/365

# Note some are close to perfect, because they are in training set and are unique in brand etc
print(f'best predictons of [{model_name}] model')
display(x_sample.sort_values(by='prediction_error_abslog').head(16).T)
print('worst predictions')
display(x_sample.sort_values(by='prediction_error_abslog').tail(16).T)
print('largest underestimate')
display(x_sample.sort_values(by='prediction_error').head(16).T)
print('largest overestimate')
display(x_sample.sort_values(by='prediction_error').tail(16).T)
print('worst prediction recent auction')
is_last_auction = x_sample.index.str.startswith('-'.join(x_sample.index[-1].split('-')[:2]))
display(x_sample[is_last_auction].sort_values(by='prediction_error_abslog').tail(8).T)

plt.figure(figsize=[8,8])
plt.plot(x_sample.age_y, x_sample.prediction_error_log, color='k', marker='s', markeredgecolor = (0, 0, 0, 0), markerfacecolor = (0, 0, 0, 1), linestyle='None', ms=1)
plt.axhline(0, lw=2, linestyle='--', color ='k')
plt.xlabel('age [years]')
plt.ylabel('prediction error [log of fraction]\n(positive: prediction overestimates)')
plt.show()

best predictons of [MLR added features] model


,2022-07-709207,2017-7-8137,2023-06-702912,2022-02-804922,2023-02-701203,2017-8-2217,2020-1-2403,2021-01-8075,2024-01-704902,2020-11-8112,2021-06-707306,2021-08-704308,2020-2-8130,2017-8-7156,2021-06-704106,2019-4-7101
brand,PEUGEOT,KIA,AUDI,TOYOTA,RENAULT,AUDI,AUDI,RENAULT,FIAT,OPEL,VOLKSWAGEN,VOLKSWAGEN,FORD,VOLKSWAGEN,VOLKSWAGEN,BMW
model,107,sorento,a7 sportback,prius,megane,a4,a7 sportback,twingo,punto,insignia sports tourer sw,polo,golf,fiesta,golf,polo,3er reihe
age,3658.0,4797.0,3802.0,3396.0,2224.0,3357.0,1640.0,4476.0,4729.0,2200.0,5570.0,2517.0,1888.0,5996.0,3438.0,6706.0
fuel,Benzine,Diesel,Diesel,Benzine/Elektriciteit,Diesel,Benzine,Benzine,Benzine,Diesel,Diesel,Benzine,Benzine,Diesel,Benzine,Diesel,Benzine
odometer,59675.0,239542.0,206074.0,260155.0,263760.0,270865.0,100946.0,143213.0,229567.0,237017.0,241293.0,160284.0,258663.0,193818.0,208851.0,218933.0
days_since_inspection_invalid,6.0,-130.0,84.0,839.0,33.0,-660.0,179.0,158.0,-239.0,-97.0,-150.0,-501.0,62.0,-771.0,150.0,102.0
age_at_import,0.0,0.0,2619.0,0.0,0.0,0.0,0.0,0.0,0.0,1189.0,0.0,756.0,0.0,0.0,0.0,1450.0
body_type,Hatchback,MPV,Hatchback,Hatchback,Hatchback,Stationwagen,Hatchback,Hatchback,Multipurpose vehicle (MPV),Stationwagen,Hatchback,Stationwagen,Hatchback,Hatchback,Hatchback,Coupe
displacement,998.0,2497.0,2967.0,1798.0,1461.0,1798.0,1984.0,1149.0,1248.0,1956.0,1390.0,1984.0,1560.0,1598.0,1199.0,2494.0
number_of_cylinders,3.0,4.0,6.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,3.0,6.0


worst predictions


,2020-9-3035,2024-03-220205,2021-09-260619,2023-12-240123,2017-8-7139,2015-03-2420,2021-12-260012,2019-4-2021,2015-02-2200,2018-8-2400,2022-09-265029,2015-03-2402,2018-7-2415,2017-5-2216,2014-12-2207,2017-3-2000
brand,PEUGEOT,MERCEDES-BENZ,LOTUS,MERCEDES-BENZ,BMW,BENTLEY,LAMBORGHINI,VOLKSWAGEN,ASTON-MARTIN,ROLLS ROYCE,LAMBORGHINI,FORD,AUSTIN-HEALEY,ALFA ROMEO,FORD,ALFA ROMEO
model,306,g63 amg,elise (lhd & rhd),g63 amg,5er reihe,contintal gt 60w12 gtc,132 se,111011,vanguish volante,phantom drophead coupe,urus,thunderbird,3000 mkiii phase ii,2000 gtv,thunderbird,2000 gtv
age,NaN,NaN,8377.0,NaN,NaN,NaN,8546.0,19787.0,NaN,NaN,NaN,21063.0,19116.0,16257.0,20973.0,16196.0
fuel,Diesel,Benzine,Benzine,Benzine,Benzine,Benzine,Benzine,Benzine,Benzine,Benzine,Benzine,Benzine,Benzine,Benzine,Benzine,Benzine
odometer,NaN,29724.0,87992.0,29724.0,362604.0,18210.0,40090.0,91157.0,4778.0,11305.0,17340.0,86207.0,106800.895872,23982.0,86207.0,23982.0
days_since_inspection_invalid,NaN,NaN,-288.0,NaN,NaN,NaN,1949.0,1099.0,NaN,NaN,NaN,NaN,-402.0,-739.0,NaN,-800.0
age_at_import,NaN,NaN,0.0,NaN,NaN,NaN,6164.0,16180.0,NaN,NaN,NaN,12826.0,18063.0,0.0,12826.0,0.0
body_type,NaN,NaN,Cabriolet,NaN,NaN,NaN,Coupe,Sedan,NaN,NaN,NaN,NaN,Cabriolet,Coupe,NaN,Coupe
displacement,NaN,NaN,1796.0,NaN,NaN,NaN,5707.0,1192.0,NaN,NaN,NaN,NaN,2912.0,NaN,NaN,NaN
number_of_cylinders,NaN,NaN,4.0,NaN,NaN,NaN,12.0,4.0,NaN,NaN,NaN,8.0,6.0,4.0,8.0,4.0


largest underestimate


,2022-09-265029,2023-12-240123,2023-02-200304,2024-03-220205,2015-02-2200,2021-12-260012,2018-8-2400,2022-05-260625,2019-11-2418,2019-4-2411,2015-03-2420,2015-01-2414,2023-11-260321,2018-6-2410,2023-02-260104,2022-05-260925
brand,LAMBORGHINI,MERCEDES-BENZ,PORSCHE,MERCEDES-BENZ,ASTON-MARTIN,LAMBORGHINI,ROLLS ROYCE,FERRARI,PORSCHE,MERCEDES-BENZ,BENTLEY,SKODA,LAMBORGHINI,MERCEDES-BENZ,LAND ROVER,LAND ROVER
model,urus,g63 amg,911 turbo s,g63 amg,vanguish volante,132 se,phantom drophead coupe,430 scuderia,panamera turbo s e-hybrid,amg s63 cabriolet,contintal gt 60w12 gtc,octavia,urus,S65 AMG,autobiography d350,range rover 3.0 lwb autobiogra
age,NaN,NaN,NaN,NaN,NaN,8546.0,NaN,NaN,NaN,NaN,NaN,315.0,435.0,NaN,NaN,NaN
fuel,Benzine,Benzine,Benzine,Benzine,Benzine,Benzine,Benzine,Benzine,Hybrid,Benzine,Benzine,Benzine,Benzine,Benzine,Diesel,Benzine
odometer,17340.0,29724.0,1285.0,29724.0,4778.0,40090.0,11305.0,11077.0,6925.0,13.0,18210.0,6796.0,35118.0,6379.0,734.0,183.0
days_since_inspection_invalid,NaN,NaN,NaN,NaN,NaN,1949.0,NaN,NaN,NaN,NaN,NaN,-2972.0,-1026.0,NaN,NaN,NaN
age_at_import,NaN,NaN,NaN,NaN,NaN,6164.0,NaN,NaN,NaN,NaN,NaN,0.0,233.0,NaN,NaN,NaN
body_type,NaN,NaN,NaN,NaN,NaN,Coupe,NaN,NaN,NaN,NaN,NaN,Sedan,MPV,NaN,NaN,NaN
displacement,NaN,NaN,NaN,NaN,NaN,5707.0,NaN,NaN,NaN,NaN,NaN,1798.0,3996.0,NaN,NaN,NaN
number_of_cylinders,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,4.0,8.0,NaN,NaN,NaN


largest overestimate


,2021-03-2206,2017-6-2400,2023-05-262310,2019-12-2407,2022-11-705411,2022-06-712506,2023-02-700604,2022-11-260311,2023-05-261909,2023-12-705424,2023-10-260320,2015-02-2202,2021-03-2610,2024-03-240106,2021-06-702006,2023-08-701216
brand,VOLVO,PORSCHE,AUDI,MERCEDES-BENZ,BMW,CHEVROLET,BMW,BMW,PORSCHE,LAND ROVER,PORSCHE,PORSCHE,LAND ROVER,LAND ROVER,PORSCHE,PORSCHE
model,xc90 t8 twin engine,panamera turbo,rs q8,amg c 63 s,x5 m,astro,x5 m,z4 m40i,panamera s e-hybrid,range rover velar,911 carrera t,panamera 4s,range rover sport,range rover sport,macan s,cayenne e-hybrid
age,579.0,2815.0,600.0,1059.0,2569.0,9428.0,2661.0,733.0,3017.0,870.0,131.0,1602.0,1851.0,1470.0,783.0,1354.0
fuel,Elektriciteit/Benzine,Benzine,Benzine/Elektriciteit,Benzine,Benzine,Benzine,Benzine,Benzine,Benzine/Elektriciteit,Benzine/Elektriciteit,Benzine,Benzine,Benzine,Benzine,Benzine,Benzine/Elektriciteit
odometer,7071.0,169156.0,19510.0,22356.0,73626.0,126008.416512,73626.0,8197.0,119785.0,45854.0,2164.0,43374.0,110979.0,48791.0,25070.0,81991.0
days_since_inspection_invalid,-882.0,-782.0,-861.0,-402.0,-432.0,370.0,-340.0,-728.0,95.0,-591.0,-1330.0,-2569.0,-121.0,9.0,-678.0,-107.0
age_at_import,0.0,2135.0,255.0,329.0,2270.0,7414.0,2270.0,0.0,0.0,125.0,0.0,645.0,1246.0,553.0,0.0,408.0
body_type,MPV,Hatchback,Hatchback,Cabriolet,Stationwagen,NaN,Stationwagen,Cabriolet,Hatchback,Stationwagen,Coupe,Hatchback,Stationwagen,Stationwagen,Stationwagen,Stationwagen
displacement,1969.0,4806.0,3996.0,3982.0,4395.0,4300.0,4395.0,2998.0,2995.0,1997.0,2981.0,4806.0,4999.0,4999.0,2995.0,2995.0
number_of_cylinders,4.0,8.0,8.0,8.0,8.0,6.0,8.0,6.0,6.0,4.0,6.0,8.0,8.0,8.0,6.0,6.0


worst prediction recent auction


,2024-03-708505,2024-03-708605,2024-03-707705,2024-03-705306,2024-03-240006,2024-03-220005,2024-03-708205,2024-03-220205
brand,MINI,PORSCHE,AUDI,BMW,MERCEDES-BENZ,MERCEDES-BENZ,MERCEDES-BENZ,MERCEDES-BENZ
model,cooper,macan s,sq5,520i,g55 amg,amg s 63 4matic,280 automatic,g63 amg
age,NaN,NaN,NaN,NaN,6740.0,NaN,19632.0,NaN
fuel,Benzine,Benzine,Diesel,Benzine,Benzine,Benzine,Benzine,Benzine
odometer,12560.0,25175.0,98892.0,234667.0,137890.0,45872.0,69832.0,29724.0
days_since_inspection_invalid,NaN,NaN,NaN,NaN,-108.0,NaN,NaN,NaN
age_at_import,NaN,NaN,NaN,NaN,5445.0,NaN,11772.0,NaN
body_type,NaN,NaN,NaN,NaN,Stationwagen,NaN,Sedan,NaN
displacement,NaN,NaN,NaN,NaN,5439.0,NaN,NaN,NaN
number_of_cylinders,NaN,NaN,NaN,NaN,8.0,NaN,6.0,NaN


## Model accuracies

In [17]:
# plot R^2

# counter for x-offset
c=0

# figure
fig = plt.figure(figsize=[4,2])
ax = fig.gca()
xs = ys = fs = [None]

# loop over all models
for name,res in models.items():

    c+=1 # x-offset

    if name == 'linear regression no cv':
        # No cv, so only one value. Make it a list of one for type consistency
        k = 'R^2'
        rsq = [res[k]]
    
    else: 
        k = 'cv R^2'
        rsq = res[k]
        
    if 'n betas effective' in res:
        ndf = res['n betas effective']
    elif 'betas' in res:
        ndf = len(res['betas'])
    elif 'n effective features' in res:
        ndf = res['n effective features']
        
    # add r-squares and offset to vectors
    ys = np.concatenate([ys, rsq])
    xs = np.concatenate([xs, np.ones_like(rsq) * c])
    fs = np.concatenate([fs, [ndf]])

# actual plotting
sns.swarmplot(x=xs, y=ys, ax=ax, hue=None)
ax.bar(range(1,len(models)+1), [res['R^2'] for res in models.values()], width=0.8, fc='none')
for x,ndf in enumerate(fs):
    if ndf is None:
        continue
    if x == 1:
        s = f'd.f.: {ndf:.0f}'
    else:
        s = f'{ndf:.0f}'
    ax.text(x, 1, s, ha='center')
# prettify
ax.set_xticks(range(1,len(models)+1))
ax.set_xticklabels(labels=list(models.keys()), rotation=45, va='top', ha='right', style='italic')
ax.set_ylim(bottom=0, top=+1)
ax.set_title('Model performance\n', style='italic')
ax.set_ylabel('Coefficient of determination\n($R^2$)', style='italic')
ax.xaxis.set_tick_params(which='minor', bottom=False)

# save
file_name = f"{RESULTS_DIR}/model-performance.png"
if True | do_save(file_name): # always save
    print(file_name)
    with plt.style.context(f"{cfg['FILE_LOCATION']['app_dir']}/assets/context-paper.mplstyle"):
        plt.savefig(file_name, bbox_inches='tight', transparent=False)
else:
    plt.show()
    print(f'Skip. {file_name} exists or saving is disabled in settings.')

/home/tom/bin/satdatsci/Saturday-Datascience/results/model-performance.png


In [18]:
# plot data

# loop over all models
for model_name in models.keys():
    print(model_name)
    res = models[model_name]
    features = num_columns.copy()
    
    # model specific adjustments
    if (model_name == 'linear regression log price') \
    or (model_name == 'linear regression log price young'):
        yX = df.loc[:,['price', 'age']].dropna()
        X = yX.iloc[:,1]
        y = yX.iloc[:,0]
        # log price is used
        y = np.log10(y)
        # unit
        unit = '(log[EUR])'
    elif (model_name == 'MLR reduced observations') \
    or (model_name == 'MLR impute median'):
        yX = df.dropna(subset=['price'] + features).loc[:,['price'] + features]
        X = yX.iloc[:,1:]
        y = np.log10(yX.iloc[:,0])
        unit = '(log[EUR])'
    elif (model_name == 'MLR with categorical') \
    or (model_name == 'MLR Lasso'):
        yX = df.dropna(subset=['price']).copy()
        X = yX.iloc[:,1:]
        y = yX.iloc[:,0]      
        unit = '(EUR)'
    elif (model_name == 'MLR added features'):
        yX = df.dropna(subset=['price']).copy()
        X = yX.iloc[:,1:]
        y = yX.iloc[:,0]      
        unit = '(EUR)'
        X.loc[:,'usage_intensity'] = X.odometer / X.age
        X.loc[:,'classic'] = X.age > 25*365
        X.loc[:,'classic'].replace({True:'y', False:'n'}, inplace=True)
    elif (model_name == 'Decision Tree Regression'):
        yX = df.dropna(subset=['price']).copy()
        X = yX.iloc[:,1:]
        y = yX.iloc[:,0]      
        unit = '(EUR)'
        X.loc[:,'usage_intensity'] = X.odometer / X.age
        X.loc[:,'classic'] = X.age > 25*365
        X.loc[:,'classic'].replace({True:+1, False:-1, np.NaN:0}, inplace=True)
        for col in ['fourwd', 'under_survey', 'automatic_gearbox']:
            X.loc[:, col].replace({'n':0, 'y':1}, inplace=True)
            
    else:
        # all original data
        yX = df.loc[:,['price', 'age']].dropna()
        X = yX.iloc[:,1]
        y = yX.iloc[:,0]
        unit = '(EUR)'
    
    if X.ndim != 1:
        n_feat = X.shape[1]
    else:
        n_feat = 1
        
    if not model_name in ('MLR with categorical', 'MLR Lasso', 'MLR added features', 'Decision Tree Regression'):
        # needed for .predict
        X = np.array(X).reshape(-1,n_feat)
        y = np.array(y).reshape(-1,1)
    
    # predict all data
    y_pred = res['model'].predict(X)
    if max(y) < 10:
        rmse = np.sqrt(np.mean(((10**y)-(10**y_pred))**2))
    else:
        rmse = np.sqrt(np.mean((y-y_pred)**2))
    print(rmse)

    # actual plotting
    fig,axs = plt.subplots(nrows=2, ncols=1, figsize=[8,8])
    
    # data
    axs[0].plot(y, y_pred, marker=',', linestyle='None')
    # error
    axs[1].plot(y, y_pred-y, marker=',', linestyle='None')
    
    # axis equal for top
    if (model_name == 'MLR with categorical') or (model_name == 'MLR Lasso') or (model_name == 'MLR added features') or (model_name == 'Decision Tree Regression'):
        axs[0].set_xscale('log')
        axs[0].set_yscale('log')
        axs[1].set_xscale('log')
    axs[0].set_aspect(1)
    # store limits
    yl = axs[0].get_ylim()
    xl_top = axs[0].get_xlim()
    xl_bot = axs[1].get_xlim()
    xl = [np.max([xl_top[0], xl_bot[0]]), np.min([xl_top[1], xl_bot[1]])]
    # plot unity line and 0 error
    unity_line = [np.max([xl[0], yl[0]]), np.min([xl[1], yl[1]])]
    axs[0].plot(unity_line, unity_line, '-k', linewidth=2)
    axs[1].plot(xl, [0, 0], '-k', linewidth=2)
    # reset limits
    axs[0].set_xlim(xl)
    axs[1].set_xlim(xl)

    # make equal size panels
    # Note: sharex did not work
    bb=axs[0].get_position(False)
    rect_top = bb.bounds
    bb=axs[1].get_position(False)
    rect_bot = bb.bounds
    rect = list(rect_bot)
    rect[0] = rect_top[0]
    rect[2] = rect_top[2]
    axs[1].set_position(rect)
    
    # labeling
    fig.suptitle('{}\nrmse: EUR {:.0f}'.format(model_name,rmse), style='italic')
    axs[1].set_xlabel('Real price ' + unit, style='italic')
    axs[0].set_ylabel('Predicted price\n' + unit, style='italic')
    axs[1].set_ylabel('Prediction error\n' + unit, style='italic')
    
    # save
    file_name = f"{RESULTS_DIR}/{model_name.replace(' ','_')}-accuracy.png"
    if True | do_save(file_name): # always save
        print(file_name)
        with plt.style.context(f"{cfg['FILE_LOCATION']['app_dir']}/assets/context-paper.mplstyle"):
            plt.savefig(file_name, bbox_inches='tight', transparent=False)
    else:
        plt.show()
        print(f'Skip. {file_name} exists or saving is disabled in settings.')

linear regression no cv
9984.073640350962
/home/tom/bin/satdatsci/Saturday-Datascience/results/linear_regression_no_cv-accuracy.png
linear regression log price young
9188.540387621768
/home/tom/bin/satdatsci/Saturday-Datascience/results/linear_regression_log_price_young-accuracy.png
MLR reduced observations
8619.659181826759
/home/tom/bin/satdatsci/Saturday-Datascience/results/MLR_reduced_observations-accuracy.png
MLR impute median
10602.550419578916
/home/tom/bin/satdatsci/Saturday-Datascience/results/MLR_impute_median-accuracy.png
MLR with categorical
9962.944195598433
/home/tom/bin/satdatsci/Saturday-Datascience/results/MLR_with_categorical-accuracy.png
MLR Lasso
7626.972659942004
/home/tom/bin/satdatsci/Saturday-Datascience/results/MLR_Lasso-accuracy.png
MLR added features
8253.948958097393
/home/tom/bin/satdatsci/Saturday-Datascience/results/MLR_added_features-accuracy.png
Decision Tree Regression
6566.703168667472
/home/tom/bin/satdatsci/Saturday-Datascience/results/Decision_Tree